Welcome to the SQL mini project. For this project, you will use Springboard' online SQL platform, which you can log into through the following link:

https://sql.springboard.com/

Username: student

Password: learn_sql@springboard

The data you need is in the "country_club" database. This database
contains 3 tables:

    i) the "Bookings" table,
    
    ii) the "Facilities" table, and
    
    iii) the "Members" table.


Before executing queries, let's create database in MySQL terminal as follows:

mysql> CREATE DATABASE country_club;

mysql> use country_club;

mysql> source country_club.sql


And connect to newly created database:

In [12]:
import mysql.connector

mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd="admin",
  database="country_club"
)

cursor = mydb.cursor()

#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [13]:
query = ("SELECT * FROM `Facilities` WHERE `membercost` > 0")

cursor.execute(query)
for x in cursor:
    print(x)

(0, 'Tennis Court 1', Decimal('5.0'), Decimal('25.0'), 10000, 200)
(1, 'Tennis Court 2', Decimal('5.0'), Decimal('25.0'), 8000, 200)
(4, 'Massage Room 1', Decimal('9.9'), Decimal('80.0'), 4000, 3000)
(5, 'Massage Room 2', Decimal('9.9'), Decimal('80.0'), 4000, 3000)
(6, 'Squash Court', Decimal('3.5'), Decimal('17.5'), 5000, 80)


#### Q2: How many facilities do not charge a fee to members? 

In [14]:
query = ("SELECT COUNT(*) FROM `Facilities` WHERE `membercost` = 0")

cursor.execute(query)
for x in cursor:
    print(x)

(4,)


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [16]:
query = ("SELECT `facid`, `name`, `membercost`, `monthlymaintenance` \
         FROM `Facilities` \
         WHERE `membercost` > 0 \
         AND `membercost` <  `monthlymaintenance` * 0.2 ")

cursor.execute(query)
for x in cursor:
    print(x)

(0, 'Tennis Court 1', Decimal('5.0'), 200)
(1, 'Tennis Court 2', Decimal('5.0'), 200)
(4, 'Massage Room 1', Decimal('9.9'), 3000)
(5, 'Massage Room 2', Decimal('9.9'), 3000)
(6, 'Squash Court', Decimal('3.5'), 80)


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [17]:
query = ("SELECT * FROM `Facilities` WHERE `facid` IN (1,5)")

cursor.execute(query)
for x in cursor:
    print(x)

(1, 'Tennis Court 2', Decimal('5.0'), Decimal('25.0'), 8000, 200)
(5, 'Massage Room 2', Decimal('9.9'), Decimal('80.0'), 4000, 3000)


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? Return the name and monthly maintenance of the facilities in question.

In [18]:
query = ("SELECT `name`, `monthlymaintenance`, \
         CASE \
             WHEN  `monthlymaintenance`> 100 \
             THEN 'expensive' \
             ELSE 'cheap' \
         END \
         FROM `Facilities`")

cursor.execute(query)
for x in cursor:
    print(x)


('Tennis Court 1', 200, 'expensive')
('Tennis Court 2', 200, 'expensive')
('Badminton Court', 50, 'cheap')
('Table Tennis', 10, 'cheap')
('Massage Room 1', 3000, 'expensive')
('Massage Room 2', 3000, 'expensive')
('Squash Court', 80, 'cheap')
('Snooker Table', 15, 'cheap')
('Pool Table', 15, 'cheap')


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [19]:
query = ("SELECT  `surname`, `firstname`,`joindate` \
         FROM `Members` \
         WHERE `joindate` = \
         (SELECT MAX(`joindate`) FROM `Members`)")

cursor.execute(query)
for x in cursor:
    print(x)

('Smith', 'Darren', '2012-09-26 18:08:45')


#### Q7: How can you produce a list of all members who have used a tennis court? Include in your output the name of the court, and the name of the member formatted as a single column. Ensure no duplicate data, and order by the member name.

In [20]:
query = ("SELECT  DISTINCT CONCAT(m.surname , ' ' ,  m.firstname) as member_name, f.name as facility_name \
         FROM Members m \
         JOIN Bookings b on b.memid = m.memid \
         JOIN Facilities f on f.facid = b.facid \
         WHERE f.name LIKE 'Tennis Court %' \
         ORDER BY member_name")

cursor.execute(query)
for x in cursor:
    print(x)

('Bader Florence', 'Tennis Court 1')
('Bader Florence', 'Tennis Court 2')
('Baker Anne', 'Tennis Court 2')
('Baker Anne', 'Tennis Court 1')
('Baker Timothy', 'Tennis Court 1')
('Baker Timothy', 'Tennis Court 2')
('Boothe Tim', 'Tennis Court 2')
('Boothe Tim', 'Tennis Court 1')
('Butters Gerald', 'Tennis Court 2')
('Butters Gerald', 'Tennis Court 1')
('Coplin Joan', 'Tennis Court 1')
('Crumpet Erica', 'Tennis Court 1')
('Dare Nancy', 'Tennis Court 2')
('Dare Nancy', 'Tennis Court 1')
('Farrell David', 'Tennis Court 1')
('Farrell David', 'Tennis Court 2')
('Farrell Jemima', 'Tennis Court 2')
('Farrell Jemima', 'Tennis Court 1')
('Genting Matthew', 'Tennis Court 1')
('GUEST GUEST', 'Tennis Court 2')
('GUEST GUEST', 'Tennis Court 1')
('Hunt John', 'Tennis Court 1')
('Hunt John', 'Tennis Court 2')
('Jones David', 'Tennis Court 1')
('Jones David', 'Tennis Court 2')
('Jones Douglas', 'Tennis Court 1')
('Joplette Janice', 'Tennis Court 1')
('Joplette Janice', 'Tennis Court 2')
('Owen Charles',

#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? Remember that guests have different costs to members (the listed costs are per half-hour 'slot'), and the guest user's ID is always 0. Include in your output the name of the facility, the name of the member formatted as a single column, and the cost. Order by descending cost, and do not use any subqueries.

In [21]:
query = (
"SELECT  CONCAT(m.surname , ' ' ,  m.firstname) as member_name, f.name as facility_name, \
CASE   \
        WHEN m.memid = 0 THEN f.guestcost * b.slots \
        ELSE f.membercost * b.slots \
    END \
as cost \
FROM Members m \
JOIN Bookings b on b.memid = m.memid \
JOIN Facilities f on f.facid = b.facid \
WHERE b.starttime LIKE ('2012-09-14 %') \
HAVING cost > 30 \
ORDER BY cost DESC")

cursor.execute(query)
for x in cursor:
    print(x)

('GUEST GUEST', 'Massage Room 2', Decimal('320.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('150.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('75.0'))
('GUEST GUEST', 'Squash Court', Decimal('70.0'))
('Farrell Jemima', 'Massage Room 1', Decimal('39.6'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))


#### Q9: This time, produce the same result as in Q8, but using a subquery

In [22]:
query = (
"SELECT  CONCAT(m.surname , ' ' ,  m.firstname) as member_name, subquery.facility_name, subquery.cost \
FROM \
(SELECT b.memid, f.name as facility_name, b.starttime, \
CASE    \
        WHEN b.memid = 0 THEN f.guestcost * b.slots \
        ELSE f.membercost * b.slots \
    END \
as cost \
FROM Bookings b \
JOIN Facilities f on f.facid = b.facid \
) as subquery \
JOIN Members m on  subquery.memid = m.memid \
WHERE subquery.starttime LIKE ('2012-09-14 %') \
HAVING subquery.cost > 30 \
ORDER BY subquery.cost DESC")

cursor.execute(query)
for x in cursor:
    print(x)

('GUEST GUEST', 'Massage Room 2', Decimal('320.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Massage Room 1', Decimal('160.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('150.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 1', Decimal('75.0'))
('GUEST GUEST', 'Tennis Court 2', Decimal('75.0'))
('GUEST GUEST', 'Squash Court', Decimal('70.0'))
('Farrell Jemima', 'Massage Room 1', Decimal('39.6'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))
('GUEST GUEST', 'Squash Court', Decimal('35.0'))


#### Q10: Produce a list of facilities with a total revenue less than 1000. The output of facility name and total revenue, sorted by revenue. Remember that there's a different cost for guests and members! 

In [23]:
query = (
"SELECT f.name as facility_name, sum( \
CASE    \
        WHEN m.memid = 0 THEN f.guestcost * b.slots \
        ELSE f.membercost * b.slots \
    END \
 ) as revenue \
FROM Members m \
JOIN Bookings b on b.memid = m.memid \
JOIN Facilities f on f.facid = b.facid \
GROUP BY facility_name \
HAVING revenue < 1000 \
ORDER BY revenue ")

cursor.execute(query)
for x in cursor:
    print(x)


('Table Tennis', Decimal('180.0'))
('Snooker Table', Decimal('240.0'))
('Pool Table', Decimal('270.0'))
